In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import tensorflow as tf

2022-09-26 11:32:08.818610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 11:32:08.988242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-26 11:32:08.988270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-26 11:32:09.015336: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 11:32:09.691932: W tensorflow/stream_executor/platform/de

In [3]:
new_braunfels_data_path = os.path.join("..","input_data","array_input","New_Braunfels.npy")
del_rio_data_path = os.path.join("..","input_data","array_input","Del_rio_data.npy")

In [4]:
del_rio_history = np.load(del_rio_data_path)
New_Braunfels_history = np.load(new_braunfels_data_path)

In [5]:
print(New_Braunfels_history.shape)

(1, 138, 64, 64)


In [6]:
def make_time_arr(history):
    time_arr = []
    for j in range(64):
        for i in range(64):
            arr = []
            for l in range(history.shape[1]):
                arr.append(history[0][l][j][i])
            time_arr.append(arr)
    return time_arr

In [7]:
new_braunfels_time_arr = make_time_arr(New_Braunfels_history)
del_rio__time_arr = make_time_arr(del_rio_history)

In [8]:
print(len(new_braunfels_time_arr[0]))
print(len(del_rio__time_arr[0]))

138
186


In [12]:
new_braunfels_data = np.array(new_braunfels_time_arr)
del_rio_data = np.array(del_rio__time_arr)
print(new_braunfels_data.shape)
print(del_rio_data.shape)

(4096, 138)
(4096, 186)


In [13]:
def LSTM_model(data):
    for i in range(0,len(data),32):
        batch = data[i:i+32]
        batch_reshaped = np.reshape(batch, (batch.shape[0], batch.shape[1], 1))
        lstm = tf.keras.layers.LSTM(10)
        output = lstm(batch_reshaped)
        
        dense_nn = tf.keras.layers.Dense(1)
        output = dense_nn(output)
        if i==0:
            prediction = np.array(output)
        else:
            prediction = np.concatenate((prediction,output), axis = 0)
    print(prediction.shape)
    return prediction

In [14]:
new_braunfels_prediction = LSTM_model(new_braunfels_data)
del_rio_prediction = LSTM_model(del_rio_data)

(4096, 1)
(4096, 1)


In [15]:
new_braunfels_prediction = np.reshape(new_braunfels_prediction,(64,64))
del_rio_prediction = np.reshape(del_rio_prediction,(64,64))